In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import time

import argparse

import rdkit
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig

import shutil
import os
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET

# Torch
import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
from torch.autograd.variable import Variable
import torch.nn.functional as F

#import multiprocessing
#from joblib import Parallel, delayed

from __future__ import print_function

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [2]:
print(torch.__version__)
!nvidia-smi

1.12.1+cu102
Sun Nov 20 13:07:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   31C    P8    14W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                           

In [3]:
torch.cuda.is_available()

True

In [4]:
!gpustat

ip-10-0-101-229  Sun Nov 20 13:07:35 2022  520.61.05
[0] Tesla T4 | 32°C,   0 % |   404 / 15360 MB |


In [5]:
import pickle
from skmultilearn.dataset import load_from_arff, load_dataset_dump
import copy
import datetime
import jsonpickle

In [6]:
import numpy as np
from scipy.sparse import lil_matrix
from sklearn.model_selection import KFold, StratifiedKFold
import pandas as pd
import copy
from itertools import chain

In [7]:
from builtins import str
from builtins import range
from builtins import object
import arff
import bz2
import numpy as np
import os
import csv
import sys
import shutil
from os import environ
from os.path import dirname
from os.path import join
from os.path import exists
from os.path import expanduser
from os.path import isdir
from os.path import splitext
from os import listdir
from os import makedirs
from scipy import sparse


In [8]:
# Load data

In [9]:
dat_label = pd.read_csv('dataset.csv',index_col=0)
#label = label.iloc[:12,:]
dat_smiles = pd.read_csv('smiles.csv', index_col=0)
s = dat_smiles.to_dict()
dict_smile = s['SMILES']

In [10]:
dat_labels = dat_label.T

del 11980947,5287407,24238,44475014,62074,11686063,133082064,16211610,173849,CID23308253,24753271

In [11]:
def szymanski_ts_eq_fold(n_splits, y):

    y_train = lil_matrix(y)

    n_samples = y_train.shape[0] #593
    n_labels = y_train.shape[1] #6

    percentage_per_fold = [1/float(n_splits) for i in range(n_splits)]
    desired_samples_per_fold = np.array([percentage_per_fold[i]*n_samples for i in range(n_splits)]) #59.3

    folds = [[] for i in range(n_splits)] #10 lists

    samples_with_label = [[] for i in range(n_labels)]

    for sample, labels in enumerate(y_train.rows):
        for label in labels:
            samples_with_label[label].append(sample)
    # labelpair based sample size
            
    samples_with_labelpairs = {}
    for row, labels in enumerate(y_train.rows):
        pairs = [(a, b) for b in labels for a in labels if a <= b]
        for p in pairs:
            if p not in samples_with_labelpairs:
                samples_with_labelpairs[p] = []
            samples_with_labelpairs[p].append(row)

    desired_samples_per_labelpair_per_fold = {k : [len(v)*i for i in percentage_per_fold] for k,v in samples_with_labelpairs.items()}

    labels_of_edges = samples_with_labelpairs.keys() # 20 pairs
    labeled_samples_available = [len(samples_with_labelpairs[v]) for v in labels_of_edges] #XXXXXX
    # labelpair based sample size
    
    rows_used = {i : False for i in range(n_samples)}
    total_labeled_samples_available = sum(labeled_samples_available) #1723
    old_l=None

    while total_labeled_samples_available > 0:
        l = list(labels_of_edges)[np.argmin(np.ma.masked_equal(labeled_samples_available, 0, copy=False))]

        while len(samples_with_labelpairs[l])>0:

            row = samples_with_labelpairs[l].pop()
            if rows_used[row]:
                continue

            max_val = max(desired_samples_per_labelpair_per_fold[l])
            M = np.where(np.array(desired_samples_per_labelpair_per_fold[l])==max_val)[0]
            #print(l, M, len(M))

            m = None
            if len(M) == 1:
                m = M[0]
            else:
                max_val = max(desired_samples_per_fold[M])
                M_bis = np.where(np.array(desired_samples_per_fold)==max_val)[0]
                M_bis = np.array([x for x in M_bis if x in M])
                m = np.random.choice(M_bis, 1)[0]

            folds[m].append(row)
            rows_used[row]=True #----
            desired_samples_per_labelpair_per_fold[l][m]-=1
            if desired_samples_per_labelpair_per_fold[l][m] <0:
                desired_samples_per_labelpair_per_fold[l][m]=0

            for i in samples_with_labelpairs.keys():
                if row in samples_with_labelpairs[i]:
                    samples_with_labelpairs[i].remove(row)
                    desired_samples_per_labelpair_per_fold[i][m]-=1

                if desired_samples_per_labelpair_per_fold[i][m] <0:
                    desired_samples_per_labelpair_per_fold[i][m]=0
            desired_samples_per_fold[m]-=1

        labeled_samples_available = [len(samples_with_labelpairs[v]) for v in labels_of_edges]
        total_labeled_samples_available = sum(labeled_samples_available)

        available_samples = [i for i, v in rows_used.items() if not v]
        samples_left = len(available_samples)


    assert (samples_left + sum(map(len, folds))) == n_samples

    while samples_left>0:
        row = available_samples.pop()
        rows_used[row]=True
        fold_selected = np.random.choice(np.where(desired_samples_per_fold>0)[0], 1)[0]
        folds[fold_selected].append(row)
        samples_left-=1

    assert sum(map(len, folds)) == n_samples
    assert len([i for i, v in rows_used.items() if not v])==0
    return folds

In [12]:
import scipy
scipy.sparse.lil_matrix(dat_labels.values)

<5596x91 sparse matrix of type '<class 'numpy.int64'>'
	with 22713 stored elements in List of Lists format>

In [13]:
n_splits = 10
folds =szymanski_ts_eq_fold(n_splits, dat_labels)

In [14]:
# Parser check
def restricted_float(x, inter):
    x = float(x)
    if x < inter[0] or x > inter[1]:
        raise argparse.ArgumentTypeError("%r not in range [1e-5, 1e-4]"%(x,))
    return x

# Argument parser
import sys
sys.argv=['']
del sys

parser = argparse.ArgumentParser(description='Neural message passing')

parser.add_argument('--dataset', default='qm9', help='QM9')
parser.add_argument('--datasetPath', default='/home/taobai/Documents/Model/MPNN/mpnn-data/qm9/dsgdb9nsd/', help='dataset path')
parser.add_argument('--logPath', default='./log/qm9/mpnn/', help='log path')
parser.add_argument('--plotLr', default=False, help='allow plotting the data')
parser.add_argument('--plotPath', default='./plot/qm9/mpnn/', help='plot path')
#parser.add_argument('--resume', default='./checkpoint/qm9/mpnn/',
#                    help='path to latest checkpoint')

# Optimization Options
parser.add_argument('--batch-size', type=int, default=10, metavar='N',
                    help='Input batch size for training (default: 20)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Enables CUDA training')
parser.add_argument('--epochs', type=int, default=5, metavar='N',
                    help='Number of epochs to train (default: 100)')
parser.add_argument('--lr', type=lambda x: restricted_float(x, [1e-5, 1e-2]), default=1e-3, metavar='LR',
                    help='Initial learning rate [1e-5, 5e-4] (default: 1e-4)')
parser.add_argument('--lr-decay', type=lambda x: restricted_float(x, [.01, 1]), default=0.6, metavar='LR-DECAY',
                    help='Learning rate decay factor [.01, 1] (default: 0.6)')
parser.add_argument('--schedule', type=list, default=[0.1, 0.9], metavar='S',
                    help='Percentage of epochs to start the learning rate decay [0, 1] (default: [0.1, 0.9])')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum (default: 0.9)')
# i/o
parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                    help='How many batches to wait before logging training status')
# Accelerating
parser.add_argument('--prefetch', type=int, default=4, help='Pre-fetching threads.')

# Model modification
parser.add_argument('--model', type=str,help='MPNN model name [MPNN, MPNNv2, MPNNv3]',
                        default='MPNN')

_StoreAction(option_strings=['--model'], dest='model', nargs=None, const=None, default='MPNN', type=<class 'str'>, choices=None, help='MPNN model name [MPNN, MPNNv2, MPNNv3]', metavar=None)

In [15]:
args = parser.parse_args()
print(args)

Namespace(dataset='qm9', datasetPath='/home/taobai/Documents/Model/MPNN/mpnn-data/qm9/dsgdb9nsd/', logPath='./log/qm9/mpnn/', plotLr=False, plotPath='./plot/qm9/mpnn/', batch_size=10, no_cuda=False, epochs=5, lr=0.001, lr_decay=0.6, schedule=[0.1, 0.9], momentum=0.9, log_interval=100, prefetch=4, model='MPNN')


In [16]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [17]:
best_er1 = 0

In [18]:
def xyz_graph_reader(CID):
    smiles = dict_smile[CID]
    m = Chem.MolFromSmiles(smiles)
    m = Chem.AddHs(m)
   
    g = nx.Graph()
    # Create nodes
    for i in range(0, m.GetNumAtoms()):
        atom_i = m.GetAtomWithIdx(i)

        g.add_node(i, a_type=atom_i.GetSymbol(), a_num=atom_i.GetAtomicNum(), acceptor=0, donor=0,
                   aromatic=atom_i.GetIsAromatic(), hybridization=atom_i.GetHybridization(),
                   num_h=atom_i.GetTotalNumHs())


    # Read Edges
    for i in range(0, m.GetNumAtoms()):
        for j in range(0, m.GetNumAtoms()):
            e_ij = m.GetBondBetweenAtoms(i, j)
            if e_ij is not None:
                g.add_edge(i, j, b_type=e_ij.GetBondType())
            else:
                # Unbonded
                g.add_edge(i, j, b_type=None)

    l = list(dat_label[CID].values)           
                
    return g , l

In [19]:
def qm9_edges(g):
    remove_edges = []
    e={}    
    for n1, n2, d in g.edges(data=True):
        e_t = []
        # Raw distance function
        if d['b_type'] is None:
            remove_edges += [(n1, n2)]
        else:
            #e_t.append(d['distance'])
            e_t += [int(d['b_type'] == x) for x in [rdkit.Chem.rdchem.BondType.SINGLE, rdkit.Chem.rdchem.BondType.DOUBLE,
                                                    rdkit.Chem.rdchem.BondType.TRIPLE, rdkit.Chem.rdchem.BondType.AROMATIC]]
        if e_t:
            e[(n1, n2)] = e_t
    for edg in remove_edges:
        g.remove_edge(*edg)    
    
    return nx.to_numpy_matrix(g), e
    

In [20]:
def qm9_nodes(g, hydrogen=False):
    h = []
    for n, d in g.nodes(data=True): 
        h_t = []
        # Atom type (One-hot H, C, N, O F)
        h_t += [int(d['a_type'] == x) for x in ['H', 'C', 'N', 'O', 'F']]
        # Atomic number
        h_t.append(d['a_num'])
        # Acceptor
        h_t.append(d['acceptor'])
        # Donor
        h_t.append(d['donor'])
        # Aromatic
        h_t.append(int(d['aromatic']))
        # If number hydrogen is used as a
        if hydrogen:
            h_t.append(d['num_h'])
        h.append(h_t)
    return h

In [21]:
class Qm9():
#class Qm9(data.Dataset):
    # Constructor
    def __init__(self, idx, vertex_transform=qm9_nodes, edge_transform=qm9_edges,
                 target_transform=None, e_representation='raw_distance'):
        self.idx = idx
        self.vertex_transform = vertex_transform
        self.edge_transform = edge_transform
        self.target_transform = target_transform
        #化学分子距离度量方式，有三种，这里用的是'raw_distance'
        self.e_representation = e_representation

    def __getitem__(self,index):
        #读图，返回networkx类型的图，标签list
        #self.ids[index]是根据index得到化学分子文件名
        #self.root是路径
        g, target = xyz_graph_reader(self.idx[index])
        if self.vertex_transform is not None:
            h = self.vertex_transform(g)

        if self.edge_transform is not None:
            g, e = self.edge_transform(g)

        if self.target_transform is not None:
            target = self.target_transform(target)

        #g：邻接矩阵
        #h：每个点的特征（list of list）
        #e：词典，key是边，value是特征
        #target：标签list
        return (g, h, e), target

    def __len__(self):
        return len(self.idx)

    def set_target_transform(self, target_transform):
        self.target_transform = target_transform

In [25]:
idx = folds[2]+folds[3]+folds[4]+folds[5]+folds[6]+folds[7]+folds[8]+folds[9]
valid_ids = [dat_label.columns[i] for i in folds[0]]
test_ids = [dat_label.columns[i] for i in folds[1]]
train_ids =  [dat_label.columns[i] for i in idx]

data_train = Qm9(train_ids, edge_transform=qm9_edges, e_representation='raw_distance')
data_valid = Qm9(valid_ids, edge_transform=qm9_edges, e_representation='raw_distance')
data_test = Qm9(test_ids, edge_transform=qm9_edges, e_representation='raw_distance')

In [26]:
# Define model and optimizer

In [27]:
# Select one graph
g_tuple, l = data_train[0]
g, h_t, e = g_tuple

In [28]:
def get_values(obj, start, end):
    vals = []
    for i in range(start, end):
        vals.append(obj[i][1])
    return vals
def get_graph_stats(graph_obj_handle):
    inputs = len(graph_obj_handle)
    res = get_values(graph_obj_handle, 0, inputs) 
    param = np.array(res)
    
    stat_dict = {}
    
    stat_dict['target_mean'] = np.mean(param, axis=0)
    stat_dict['target_std'] = np.std(param, axis=0)

    return stat_dict

In [29]:
stat_dict = get_graph_stats(data_valid)

In [30]:
# set_target_transform was defined in Class Qm9
data_train.set_target_transform(lambda x:x)
data_valid.set_target_transform(lambda x: x)
data_test.set_target_transform(lambda x: x)

In [31]:
# Data Loader

In [32]:
def collate_g(batch):

    batch_sizes = np.max(np.array([[len(input_b[1]), len(input_b[1][0]), len(input_b[2]),
                                len(list(input_b[2].values())[0])]
                                if input_b[2] else
                                [len(input_b[1]), len(input_b[1][0]), 0,0]
                                for (input_b, target_b) in batch]), axis=0)

    g = np.zeros((len(batch), batch_sizes[0], batch_sizes[0]))
    h = np.zeros((len(batch), batch_sizes[0], batch_sizes[1]))
    e = np.zeros((len(batch), batch_sizes[0], batch_sizes[0], batch_sizes[3]))

    target = np.zeros((len(batch), len(batch[0][1])))

    for i in range(len(batch)):

        num_nodes = len(batch[i][0][1])

        # Adjacency matrix
        g[i, 0:num_nodes, 0:num_nodes] = batch[i][0][0]

        # Node features
        h[i, 0:num_nodes, :] = batch[i][0][1]

        # Edges
        for edge in batch[i][0][2].keys():
            e[i, edge[0], edge[1], :] = batch[i][0][2][edge]
            e[i, edge[1], edge[0], :] = batch[i][0][2][edge]

        # Target
        target[i, :] = batch[i][1]

    g = torch.FloatTensor(g)
    h = torch.FloatTensor(h)
    e = torch.FloatTensor(e)
    target = torch.FloatTensor(target)

    return g, h, e, target

In [33]:

#注意这里面的collate_g函数,在utils里面
train_loader = torch.utils.data.DataLoader(data_train,
                                           batch_size=args.batch_size,shuffle=True, collate_fn=collate_g,
                                           num_workers=args.prefetch, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(data_valid,
                                           batch_size=args.batch_size, collate_fn=collate_g,
                                           num_workers=args.prefetch, pin_memory=True)
test_loader = torch.utils.data.DataLoader(data_test,
                                          batch_size=args.batch_size, collate_fn=collate_g,
                                          num_workers=args.prefetch, pin_memory=True)

In [34]:
#define model

In [35]:
class NNet(nn.Module):

    def __init__(self, n_in, n_out, hlayers=(91, 256, 128)):
        super(NNet, self).__init__()
        self.n_hlayers = len(hlayers)
        self.fcs = nn.ModuleList([nn.Linear(n_in, hlayers[i]) if i == 0 else
                                  nn.Linear(hlayers[i-1], n_out) if i == self.n_hlayers else
                                  nn.Linear(hlayers[i-1], hlayers[i]) for i in range(self.n_hlayers+1)])

    def forward(self, x):
        x = x.contiguous().view(-1, self.num_flat_features(x))
        for i in range(self.n_hlayers):
            x = F.relu(self.fcs[i](x))
        x = self.fcs[-1](x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [36]:
#dtype = torch.cuda.FloatTensor
dtype = torch.FloatTensor

In [37]:
class MessageFunction(nn.Module):

    # Constructor
    def __init__(self, message_def='mpnn', args={}):
        super(MessageFunction, self).__init__()
        self.m_definition = ''
        self.m_function = None
        self.args = {}
        self.__set_message(message_def, args)

    # Message from h_v to h_w through e_vw
    def forward(self, h_v, h_w, e_vw, args=None):
        return self.m_function(h_v, h_w, e_vw, args)

    # Set a message function
    def __set_message(self, message_def, args={}):
        self.m_definition = message_def.lower()

        self.m_function = {
                    'duvenaud':         self.m_duvenaud,
                    'intnet':             self.m_intnet,
                    'mpnn':             self.m_mpnn,
                }.get(self.m_definition, None)

        if self.m_function is None:
            print('WARNING!: Message Function has not been set correctly\n\tIncorrect definition ' + message_def)
            quit()

        init_parameters = {
            'duvenaud': self.init_duvenaud,            
            'intnet':     self.init_intnet,
            'mpnn':     self.init_mpnn
        }.get(self.m_definition, lambda x: (nn.ParameterList([]), nn.ModuleList([]), {}))

        self.learn_args, self.learn_modules, self.args = init_parameters(args)

        self.m_size = {
                'duvenaud':     self.out_duvenaud,            
                'intnet':         self.out_intnet,
                'mpnn':         self.out_mpnn
            }.get(self.m_definition, None)

    # Get the name of the used message function
    def get_definition(self):
        return self.m_definition

    # Get the message function arguments
    def get_args(self):
        return self.args

    # Get Output size
    def get_out_size(self, size_h, size_e, args=None):
        return self.m_size(size_h, size_e, args)
    
    
    # Duvenaud et al. (2015), Convolutional Networks for Learning Molecular Fingerprints
    def m_duvenaud(self, h_v, h_w, e_vw, args):
        m = torch.cat([h_w, e_vw], 2)
        return m

    def out_duvenaud(self, size_h, size_e, args):
        return size_h + size_e

    def init_duvenaud(self, params):
        learn_args = []
        learn_modules = []
        args = {}
        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args 

    # Battaglia et al. (2016), Interaction Networks
    def m_intnet(self, h_v, h_w, e_vw, args):
        m = torch.cat([h_v[:, None, :].expand_as(h_w), h_w, e_vw], 2)
        b_size = m.size()

        m = m.view(-1, b_size[2])

        m = self.learn_modules[0](m)
        m = m.view(b_size[0], b_size[1], -1)
        return m

    def out_intnet(self, size_h, size_e, args):
        return self.args['out']

    def init_intnet(self, params):
        learn_args = []
        learn_modules = []
        args = {}
        args['in'] = params['in']
        args['out'] = params['out']
        learn_modules.append(NNet(n_in=params['in'], n_out=params['out']))
        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args

    # Gilmer et al. (2017), Neural Message Passing for Quantum Chemistry
    def m_mpnn(self, h_v, h_w, e_vw, opt={}):
        # Matrices for each edge
        edge_output = self.learn_modules[0](e_vw)
        edge_output = edge_output.view(-1, self.args['out'], self.args['in'])

        #h_w_rows = h_w[..., None].expand(h_w.size(0), h_v.size(1), h_w.size(1)).contiguous()
        h_w_rows = h_w[..., None].expand(h_w.size(0), h_w.size(1), h_v.size(1)).contiguous()

        h_w_rows = h_w_rows.view(-1, self.args['in'])

        h_multiply = torch.bmm(edge_output, torch.unsqueeze(h_w_rows,2))

        m_new = torch.squeeze(h_multiply)

        return m_new

    def out_mpnn(self, size_h, size_e, args):
        return self.args['out']

    def init_mpnn(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        args['in'] = params['in']
        args['out'] = params['out']

        # Define a parameter matrix A for each edge label.
        learn_modules.append(NNet(n_in=params['edge_feat'], n_out=(params['in']*params['out'])))

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args

In [38]:
# UpdateFunction.py: Updates the nodes using the previous state and the message.

class UpdateFunction(nn.Module):

    # Constructor
    def __init__(self, update_def='nn', args={}):
        super(UpdateFunction, self).__init__()
        self.u_definition = ''
        self.u_function = None
        self.args = {}
        self.__set_update(update_def, args)

    # Update node hv given message mv
    def forward(self, h_v, m_v, opt={}):
        return self.u_function(h_v, m_v, opt)

    # Set update function
    def __set_update(self, update_def, args):
        self.u_definition = update_def.lower()

        self.u_function = {
                    'duvenaud':         self.u_duvenaud,            
                    'intnet':             self.u_intnet,
                    'mpnn':             self.u_mpnn
                }.get(self.u_definition, None)

        if self.u_function is None:
            print('WARNING!: Update Function has not been set correctly\n\tIncorrect definition ' + update_def)

        init_parameters = {
            'duvenaud':         self.init_duvenaud,            
            'intnet':             self.init_intnet,
            'mpnn':             self.init_mpnn
        }.get(self.u_definition, lambda x: (nn.ParameterList([]), nn.ModuleList([]), {}))

        self.learn_args, self.learn_modules, self.args = init_parameters(args)

    # Get the name of the used update function
    def get_definition(self):
        return self.u_definition

    # Get the update function arguments
    def get_args(self):
        return self.args
    
    # Duvenaud
    def u_duvenaud(self, h_v, m_v, opt):

        param_sz = self.learn_args[0][opt['deg']].size()
        parameter_mat = torch.t(self.learn_args[0][opt['deg']])[None, ...].expand(m_v.size(0), param_sz[1], param_sz[0])
        
        #print(parameter_mat.size())
        #print(m_v.size())
        #print(torch.transpose(m_v.unsqueeze(-2), 1, 2).size())

        #aux = torch.bmm(parameter_mat, torch.transpose(m_v, 1, 2))
        aux = torch.bmm(parameter_mat, torch.transpose(m_v.unsqueeze(-2), 1, 2))

        return torch.transpose(torch.nn.Sigmoid()(aux), 1, 2)

    def init_duvenaud(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        # Filter degree 0 (the message will be 0 and therefore there is no update
        args['deg'] = [i for i in params['deg'] if i!=0]
        args['in'] = params['in']
        args['out'] = params['out']

        # Define a parameter matrix H for each degree.
        learn_args.append(torch.nn.Parameter(torch.randn(len(args['deg']), args['in'], args['out'])))

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args    

    # Battaglia et al. (2016), Interaction Networks
    def u_intnet(self, h_v, m_v, opt):
        if opt['x_v'].ndimension():
            input_tensor = torch.cat([h_v, opt['x_v'], torch.squeeze(m_v)], 1)
        else:
            input_tensor = torch.cat([h_v, torch.squeeze(m_v)], 1)

        return self.learn_modules[0](input_tensor)

    def init_intnet(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        args['in'] = params['in']
        args['out'] = params['out']

        learn_modules.append(NNet(n_in=params['in'], n_out=params['out']))

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args

    def u_mpnn(self, h_v, m_v, opt={}):
        h_in = h_v.view(-1,h_v.size(2))
        m_in = m_v.view(-1,m_v.size(2))
        h_new = self.learn_modules[0](m_in[None,...],h_in[None,...])[0] # 0 or 1???
        return torch.squeeze(h_new).view(h_v.size())

    def init_mpnn(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        args['in_m'] = params['in_m']
        args['out'] = params['out']

        # GRU
        learn_modules.append(nn.GRU(params['in_m'], params['out']))

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args

In [39]:
#ReadoutFunction
#dtype = torch.cuda.FloatTensor
dtype = torch.FloatTensor

class ReadoutFunction(nn.Module):

    # Constructor
    def __init__(self, readout_def='nn', args={}):
        super(ReadoutFunction, self).__init__()
        self.r_definition = ''
        self.r_function = None
        self.args = {}
        self.__set_readout(readout_def, args)

    # Readout graph given node values at las layer
    def forward(self, h_v):
        return self.r_function(h_v)

    # Set a readout function
    def __set_readout(self, readout_def, args):
        self.r_definition = readout_def.lower()

        self.r_function = {
                    'duvenaud': self.r_duvenaud,            
                    'intnet':     self.r_intnet,
                    'mpnn':     self.r_mpnn
                }.get(self.r_definition, None)

        if self.r_function is None:
            print('WARNING!: Readout Function has not been set correctly\n\tIncorrect definition ' + readout_def)
            quit()

        init_parameters = {
            'duvenaud': self.init_duvenaud,            
            'intnet':     self.init_intnet,
            'mpnn':     self.init_mpnn
        }.get(self.r_definition, lambda x: (nn.ParameterList([]), nn.ModuleList([]), {}))

        self.learn_args, self.learn_modules, self.args = init_parameters(args)

    # Get the name of the used readout function
    def get_definition(self):
        return self.r_definition

    # Duvenaud
    def r_duvenaud(self, h):
        # layers
        aux = []
        for l in range(len(h)):
            param_sz = self.learn_args[l].size()
            parameter_mat = torch.t(self.learn_args[l])[None, ...].expand(h[l].size(0), param_sz[1],
                                                                                      param_sz[0])

            aux.append(torch.transpose(torch.bmm(parameter_mat, torch.transpose(h[l], 1, 2)), 1, 2))

            for j in range(0, aux[l].size(1)):
                # Mask whole 0 vectors
                aux[l][:, j, :] = nn.Softmax()(aux[l][:, j, :].clone())*(torch.sum(aux[l][:, j, :] != 0, 1) > 0)[...,None].expand_as(aux[l][:, j, :]).type_as(aux[l])

        aux = torch.sum(torch.sum(torch.stack(aux, 3), 3), 1)
        return self.learn_modules[0](torch.squeeze(aux))

    def init_duvenaud(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        args['out'] = params['out']

        # Define a parameter matrix W for each layer.
        for l in range(params['layers']):
            learn_args.append(nn.Parameter(torch.randn(params['in'][l], params['out'])))

        # learn_modules.append(nn.Linear(params['out'], params['target']))

        learn_modules.append(NNet(n_in=params['out'], n_out=params['target']))
        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args    
    
    
    # Battaglia et al. (2016), Interaction Networks
    def r_intnet(self, h):

        aux = torch.sum(h[-1],1)

        return self.learn_modules[0](aux)

    def init_intnet(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        learn_modules.append(NNet(n_in=params['in'], n_out=params['target']))

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args

    def r_mpnn(self, h):

        aux = Variable( torch.Tensor(h[0].size(0), self.args['out']).type_as(h[0].data).zero_() )
        # For each graph
        for i in range(h[0].size(0)):
            nn_res = nn.Sigmoid()(self.learn_modules[0](torch.cat([h[0][i,:,:], h[-1][i,:,:]], 1)))*self.learn_modules[1](h[-1][i,:,:])

            # Delete virtual nodes
            nn_res = (torch.sum(h[0][i,:,:],1)[...,None].expand_as(nn_res)>0).type_as(nn_res)* nn_res

            aux[i,:] = torch.sum(nn_res,0)

        return aux

    def init_mpnn(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        # i
        learn_modules.append(NNet(n_in=2*params['in'], n_out=params['target']))

        # j
        learn_modules.append(NNet(n_in=params['in'], n_out=params['target']))

        args['out'] = params['target']

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args


In [40]:
#!/usr/bin/python

class MPNN(nn.Module):
    """
        MPNN as proposed by Gilmer et al..

        This class implements the whole Gilmer et al. model following the functions Message, Update and Readout.

        Parameters
        ----------
        in_n : int list
            Sizes for the node and edge features.
        hidden_state_size : int
            Size of the hidden states (the input will be padded with 0's to this size).
        message_size : int
            Message function output vector size.
        n_layers : int
            Number of iterations Message+Update (weight tying).
        l_target : int
            Size of the output.
        type : str (Optional)
            Classification | [Regression (default)]. If classification, LogSoftmax layer is applied to the output vector.
    """

    def __init__(self, in_n, hidden_state_size, message_size, n_layers, l_target, type='regression'):
        super(MPNN, self).__init__()

        # Define message
        self.m = nn.ModuleList(
            [MessageFunction('mpnn', args={'edge_feat': in_n[1], 'in': hidden_state_size, 
                                           'out': message_size})])

        # Define Update
        self.u = nn.ModuleList([UpdateFunction('mpnn',args={'in_m': message_size,
                                                            'out': hidden_state_size})])

        # Define Readout
        self.r = ReadoutFunction('mpnn',args={'in': hidden_state_size,
                                              'target': l_target})

        self.type = type

        self.args = {}
        self.args['out'] = hidden_state_size

        self.n_layers = n_layers

    def forward(self, g, h_in, e):

        h = []

        # Padding to some larger dimension d
        h_t = torch.cat([h_in, Variable(
            torch.zeros(h_in.size(0), h_in.size(1), self.args['out'] - h_in.size(2)).type_as(h_in.data))], 2)

        h.append(h_t.clone())

        # Layer
        for t in range(0, self.n_layers):
            e_aux = e.view(-1, e.size(3))

            h_aux = h[t].view(-1, h[t].size(2))

            m = self.m[0].forward(h[t], h_aux, e_aux)
            m = m.view(h[0].size(0), h[0].size(1), -1, m.size(1))

            # Nodes without edge set message to 0
            m = torch.unsqueeze(g, 3).expand_as(m) * m

            m = torch.squeeze(torch.sum(m, 1))

            h_t = self.u[0].forward(h[t], m)

            # Delete virtual nodes
            h_t = (torch.sum(h_in, 2)[..., None].expand_as(h_t) > 0).type_as(h_t) * h_t
            h.append(h_t)

        # Readout
        res = self.r.forward(h)
        if self.type == 'classification':
            res = torch.sigmoid(res)
        
        return res

In [41]:
#Create model

In [42]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [43]:
from sklearn.utils.class_weight import compute_class_weight
lst_imbalance_ratio = []
for i in dat_label.values:
    class_weights=compute_class_weight('balanced',classes=np.unique(i), y=i)
    #print(np.bincount(i))
    #print(class_weights)
    #ratio = class_weights[1]/class_weights[0]
    ratio = class_weights[1]
    lst_imbalance_ratio.append(ratio)

import math
weights = []
for i in range(len(lst_imbalance_ratio)):
    weights.append(math.log(1+lst_imbalance_ratio[i]))
weights = torch.tensor(weights)

In [44]:
#点的特征维度，边的特征维度
in_n = [len(h_t[0]), len(list(e.values())[0])] 
#hidden state/embedding维度
hidden_state_size = 20
#邻居消息m_i维度（聚合后的维度）后面都用d_v表示
message_size = 20
#GNN层数
n_layers = 3
#labels数量
l_target = len(l)
#回归任务
type ='classification'
#type ='regression'

#定义mpnn模型
model = MPNN(in_n, hidden_state_size, message_size, n_layers, l_target, type=type)

del in_n, hidden_state_size, message_size, n_layers, l_target, type

#print('Optimizer')
optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)


#回归任务使用MSE 1ose
criterion = torch.nn.CrossEntropyLoss(weight=weights,reduction='mean')

#评估指标，|a-b|/|b|
#evaluation = lambda output, target: torch.mean(torch.abs(output - target) / torch.abs(target))
evaluation = lambda output, target: torch.eq(torch.round(output),target).float().mean()


lr_step = (args.lr-args.lr*args.lr_decay)/(args.epochs*args.schedule[1] - args.epochs*args.schedule[0])

In [45]:
def train(train_loader, model, criterion, optimizer, epoch, logger=None):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()
    
    list_loss = []

    # switch to train mode
    model.train()

    end = time.time()
    for i, (g, h, e, target) in enumerate(train_loader):

        # Prepare input data
        if args.cuda:
            g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Measure data loading time
        data_time.update(time.time() - end)

        optimizer.zero_grad()

        # Compute output
        output = model(g, h, e)
        train_loss = criterion(output, target)
        list_loss.append(train_loss.item())
        # Logs
        losses.update(train_loss.item(), g.size(0))
        accuracy.update(evaluation(output, target).item(), g.size(0))
        
        # compute gradient and do SGD step
        train_loss.backward()
        optimizer.step()

        # Measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.log_interval == 0 and i > 0:

            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Accuracy {err.val:.4f} ({err.avg:.4f})'
                  .format(epoch, i, len(train_loader), batch_time=batch_time,
                          data_time=data_time, loss=losses, err=accuracy))

    print('Epoch: [{0}] Accuracy {err.avg:.3f}; Average Loss {loss.avg:.3f}; Avg Time x Batch {b_time.avg:.3f}'
          .format(epoch, err=accuracy, loss=losses, b_time=batch_time))
    
    return list_loss

In [46]:
def validate(val_loader, model, criterion, evaluation, logger=None):
    batch_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (g, h, e, target) in enumerate(val_loader):

        # Prepare input data
        if args.cuda:
            g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Compute output
        output = model(g, h, e)

        # Logs
        losses.update(criterion(output, target).item(), g.size(0))
        accuracy.update(evaluation(output, target).item(), g.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

    print(' * Average accuracy {err.avg:.3f}; '
          .format(err=accuracy))


    return accuracy.avg

In [47]:
print('Check cuda')
if args.cuda:
    print('\t* Cuda')
    model = model.cuda()
    criterion = criterion.cuda()
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                state[k] = v.cuda()


# Epoch for loop
for epoch in range(0, args.epochs):
    torch.cuda.empty_cache()

    if epoch > args.epochs * args.schedule[0] and epoch < args.epochs * args.schedule[1]:
        args.lr -= lr_step
        for param_group in optimizer.param_groups:
            param_group['lr'] = args.lr

    # train for one epoch
    lst_loss = train(train_loader, model, criterion, optimizer, epoch)
    # evaluate on test set
    er1 = validate(valid_loader, model, criterion, evaluation)
    
    is_best = er1 > best_er1
    best_er1 = min(er1, best_er1)
    

Check cuda
	* Cuda
Epoch: [0][100/449]	Time 0.037 (0.047)	Data 0.009 (0.016)	Loss 43.0878 (36.9493)	Accuracy 0.7363 (0.6685)
Epoch: [0][200/449]	Time 0.476 (0.055)	Data 0.013 (0.019)	Loss 32.5630 (36.3721)	Accuracy 0.7659 (0.7089)
Epoch: [0][300/449]	Time 0.061 (0.054)	Data 0.020 (0.019)	Loss 39.0173 (36.6581)	Accuracy 0.7582 (0.7327)
Epoch: [0][400/449]	Time 0.035 (0.056)	Data 0.007 (0.020)	Loss 22.9888 (36.7127)	Accuracy 0.7681 (0.7388)
Epoch: [0] Accuracy 0.743; Average Loss 36.896; Avg Time x Batch 0.054
 * Average accuracy 0.782; 
Epoch: [1][100/449]	Time 0.112 (0.057)	Data 0.058 (0.021)	Loss 28.9061 (36.9005)	Accuracy 0.7703 (0.7752)
Epoch: [1][200/449]	Time 0.040 (0.055)	Data 0.007 (0.020)	Loss 37.2949 (36.2939)	Accuracy 0.7703 (0.7726)
Epoch: [1][300/449]	Time 0.050 (0.053)	Data 0.025 (0.019)	Loss 36.1708 (36.6058)	Accuracy 0.7802 (0.7711)
Epoch: [1][400/449]	Time 0.045 (0.055)	Data 0.008 (0.020)	Loss 37.2114 (36.5785)	Accuracy 0.7451 (0.7718)
Epoch: [1] Accuracy 0.771; Average

In [48]:
# For testing
report = []
all_pred = []
all_target =[]
batch_time = AverageMeter()
data_time = AverageMeter()
losses = AverageMeter()
accuracy = AverageMeter()

if args.cuda:
    print('\t* Cuda')
    model = model.cuda()
    criterion = criterion.cuda()
for i, (g, h, e, target) in enumerate(test_loader):
    torch.cuda.empty_cache()

    # Prepare input data
    if args.cuda:
        g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
    g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

    # Compute output
    output =model(g, h, e)
    preds = torch.round(output)
    
    all_pred.append(preds.cpu().detach().numpy())
    all_target.append(target.cpu().detach().numpy())

	* Cuda


In [49]:
all_pred = np.vstack(all_pred)
all_target = np.vstack(all_target)

In [50]:
roc_auc_score(all_target, all_pred)

0.6137538101492864

In [51]:
for i in range(91):
    print(roc_auc_score(np.transpose(all_target)[i], np.transpose(all_pred)[i]))

0.6022206959706959
0.7469755469755469
0.6513295099061522
0.8402173913043478
0.6674588086185044
0.5
0.5636281588447654
0.7269942067736185
0.6962218159658744
0.538370210469509
0.5
0.6921389396709324
0.6911167157068796
0.6180173992673993
0.5977252561103492
0.5
0.5855998591673268
0.6888489208633094
0.6854568854568854
0.8621659634317862
0.6149886511024644
0.5956937799043063
0.5
0.7629104108232965
0.5
0.47475041194145584
0.5
0.4918181818181818
0.5
0.5635131471943616
0.5
0.6837643541050354
0.7011846405228759
0.6199759720401923
0.6269349845201238
0.5055710306406686
0.9316939890710382
0.5
0.5
0.5
0.5212630506748154
0.624400479616307
0.6100812435545943
0.6985374771480803
0.5
0.5515920129519698
0.7096774193548386
0.5
0.7793764988009592
0.5748626373626373
0.5
0.5
0.6276767807465873
0.4936708860759494
0.5
0.5957564575645756
0.6427216047709408
0.6931039118161044
0.6253205128205128
0.9293577981651376
0.5
0.5
0.5
0.6787822878228782
0.7656916514320535
0.5
0.6482342007434945
0.5
0.5
0.4955116696588869
0

In [ ]:
cl = classification_report(all_target, all_pred,output_dict=True)
report = pd.DataFrame.from_dict(cl, orient='index')

In [53]:
report

,precision,recall,f1-score,support
0,0.078125,0.312500,0.125000,16
1,0.085106,0.571429,0.148148,7
2,0.080000,0.428571,0.134831,14
3,0.108108,0.800000,0.190476,10
4,0.097345,0.916667,0.176000,36
...,...,...,...,...
90,0.181360,0.935065,0.303797,77
micro avg,0.130152,0.756108,0.222077,2251
macro avg,0.076969,0.473630,0.122309,2251
weighted avg,0.160144,0.756108,0.253855,2251


In [56]:
report.to_csv("report1.csv")